In [1]:
%pip install kagglehub


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yiweilu2033/well-documented-alzheimers-dataset")

print("Path to dataset files:", path)

/Users/srinivasankarunakaran/Documents/Projects/ML-Alzheimers/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/srinivasankarunakaran/.cache/kagglehub/datasets/yiweilu2033/well-documented-alzheimers-dataset/versions/2


In [3]:
import pandas as pd

# Load the Excel file
file_path = '/Users/srinivasankarunakaran/.cache/kagglehub/datasets/yiweilu2033/well-documented-alzheimers-dataset/versions/2/oasis_cross-sectional-5708aa0a98d82080 (1).xlsx'
df = pd.read_excel(file_path)

# Display the first few rows of the dataframe
df.head()

,ID,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
0,OAS1_0001_MR1,F,R,74,2.0,3.0,29.0,0.0,1344,0.743,1.306,NaN
1,OAS1_0002_MR1,F,R,55,4.0,1.0,29.0,0.0,1147,0.810,1.531,NaN
2,OAS1_0003_MR1,F,R,73,4.0,3.0,27.0,0.5,1454,0.708,1.207,NaN
3,OAS1_0004_MR1,M,R,28,NaN,NaN,NaN,NaN,1588,0.803,1.105,NaN
4,OAS1_0005_MR1,M,R,18,NaN,NaN,NaN,NaN,1737,0.848,1.010,NaN


In [4]:
# Preprocess the data
# Fill missing values
df['SES'].fillna(df['SES'].median(), inplace=True)
df['MMSE'].fillna(df['MMSE'].mean(), inplace=True)

# Drop rows with missing 'CDR'
df.dropna(subset=['CDR'], inplace=True)

# Encode 'M/F' column
df['M/F'] = df['M/F'].apply(lambda x: 1 if x == 'M' else 0)

# Select features and target
features = ['M/F', 'Age', 'Educ', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF']
target = 'CDR'

X = df[features]
y = df[target]

# Binarize the target variable (0 for non-demented, 1 for demented)
y = y.apply(lambda x: 0 if x == 0 else 1)

/var/folders/81/qh_7c3n13xj3wn834lv9_qrw0000gp/T/ipykernel_7176/1212062878.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SES'].fillna(df['SES'].median(), inplace=True)
/var/folders/81/qh_7c3n13xj3wn834lv9_qrw0000gp/T/ipykernel_7176/1212062878.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train a RandomForestClassifier
model_csv = RandomForestClassifier(n_estimators=100, random_state=42)
model_csv.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model_csv.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Export the model
joblib.dump(model_csv, 'alzheimers_csv_model.joblib')
print("CSV model exported as alzheimers_csv_model.joblib")

Accuracy: 0.8297872340425532
              precision    recall  f1-score   support

           0       0.81      0.93      0.86        27
           1       0.88      0.70      0.78        20

    accuracy                           0.83        47
   macro avg       0.84      0.81      0.82        47
weighted avg       0.84      0.83      0.83        47

CSV model exported as alzheimers_csv_model.joblib


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

# Define paths
base_dir = '/Users/srinivasankarunakaran/.cache/kagglehub/datasets/yiweilu2033/well-documented-alzheimers-dataset/versions/2'
classes = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']
image_files = []
labels = []

# Correct path for image classes
class_paths = {
    'NonDemented': os.path.join(base_dir, 'NonDemented (2)', 'NonDemented'),
    'VeryMildDemented': os.path.join(base_dir, 'VeryMildDemented', 'VeryMildDemented'),
    'MildDemented': os.path.join(base_dir, 'MildDemented', 'MildDemented'),
    'ModerateDemented': os.path.join(base_dir, 'ModerateDemented', 'ModerateDemented')
}


for i, cls in enumerate(classes):
    cls_path = class_paths[cls]
    if os.path.isdir(cls_path):
        cls_files = [os.path.join(cls_path, f) for f in os.listdir(cls_path) if f.endswith('.png')]
        # Limit to 1000 images per class
        cls_files = cls_files[:1000]
        image_files.extend(cls_files)
        labels.extend([cls] * len(cls_files))  # Use class names instead of numeric labels

# Stratified split
train_files, val_files, train_labels, val_labels = train_test_split(
    image_files, labels, test_size=0.2, random_state=42, stratify=labels
)

# Create dataframes for the generators
train_df = pd.DataFrame({'filename': train_files, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_files, 'class': val_labels})

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='class',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the model
model_image = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile the model
model_image.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Train the model
history = model_image.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Export the model
model_image.save('alzheimers_image_model.h5')
print("Image model exported as alzheimers_image_model.h5")

In [ ]:
%pip install pandas openpyxl scikit-learn tensorflow

In [ ]:
# Install additional packages for visualization and anomaly detection
%pip install matplotlib seaborn plotly

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# Load data again for visualization
file_path = '/Users/srinivasankarunakaran/.cache/kagglehub/datasets/yiweilu2033/well-documented-alzheimers-dataset/versions/2/oasis_cross-sectional-5708aa0a98d82080 (1).xlsx'
df_viz = pd.read_excel(file_path)

print("Dataset Shape:", df_viz.shape)
print("\nDataset Info:")
print(df_viz.info())
print("\nFirst 5 rows:")
print(df_viz.head())
print("\nMissing values:")
print(df_viz.isnull().sum())
print("\nTarget variable distribution:")
print(df_viz['CDR'].value_counts())

In [ ]:
# 1. Distribution of Target Variable (CDR)
plt.figure(figsize=(15, 12))

# Subplot 1: CDR Distribution
plt.subplot(3, 3, 1)
cdr_counts = df_viz['CDR'].value_counts()
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
plt.pie(cdr_counts.values, labels=cdr_counts.index, autopct='%1.1f%%', colors=colors)
plt.title('CDR Distribution (Dementia Severity)')

# Subplot 2: Age Distribution by CDR
plt.subplot(3, 3, 2)
for cdr in sorted(df_viz['CDR'].unique()):
    age_data = df_viz[df_viz['CDR'] == cdr]['Age']
    plt.hist(age_data, alpha=0.7, label=f'CDR {cdr}', bins=15)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution by CDR')
plt.legend()

# Subplot 3: Gender Distribution by CDR
plt.subplot(3, 3, 3)
gender_cdr = pd.crosstab(df_viz['M/F'], df_viz['CDR'])
gender_cdr.plot(kind='bar', ax=plt.gca())
plt.title('Gender Distribution by CDR')
plt.xlabel('Gender (F/M)')
plt.ylabel('Count')
plt.xticks(rotation=0)

# Subplot 4: MMSE Score Distribution
plt.subplot(3, 3, 4)
df_viz['MMSE'].hist(bins=20, alpha=0.7, color='skyblue')
plt.xlabel('MMSE Score')
plt.ylabel('Frequency')
plt.title('MMSE Score Distribution')

# Subplot 5: Education Level vs CDR
plt.subplot(3, 3, 5)
sns.boxplot(data=df_viz, x='CDR', y='Educ')
plt.title('Education Level by CDR')
plt.xlabel('CDR')
plt.ylabel('Education Years')

# Subplot 6: Brain Volume (eTIV) Distribution
plt.subplot(3, 3, 6)
sns.violinplot(data=df_viz, x='CDR', y='eTIV')
plt.title('Estimated Total Intracranial Volume by CDR')
plt.xlabel('CDR')
plt.ylabel('eTIV')

# Subplot 7: Normalized Whole Brain Volume
plt.subplot(3, 3, 7)
sns.boxplot(data=df_viz, x='CDR', y='nWBV')
plt.title('Normalized Whole Brain Volume by CDR')
plt.xlabel('CDR')
plt.ylabel('nWBV')

# Subplot 8: SES Distribution
plt.subplot(3, 3, 8)
df_viz['SES'].hist(bins=10, alpha=0.7, color='lightcoral')
plt.xlabel('Socioeconomic Status')
plt.ylabel('Frequency')
plt.title('SES Distribution')

# Subplot 9: Atlas Scaling Factor
plt.subplot(3, 3, 9)
sns.scatterplot(data=df_viz, x='Age', y='ASF', hue='CDR')
plt.title('Atlas Scaling Factor vs Age')
plt.xlabel('Age')
plt.ylabel('ASF')

plt.tight_layout()
plt.show()

In [ ]:
# 2. Correlation Matrix and Feature Relationships
plt.figure(figsize=(15, 10))

# Correlation Heatmap
plt.subplot(2, 3, 1)
# Select only numeric columns for correlation
numeric_cols = df_viz.select_dtypes(include=[np.number]).columns
correlation_matrix = df_viz[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Age vs MMSE relationship
plt.subplot(2, 3, 2)
sns.scatterplot(data=df_viz, x='Age', y='MMSE', hue='CDR', alpha=0.7)
plt.title('Age vs MMSE Score by CDR')
plt.xlabel('Age')
plt.ylabel('MMSE Score')

# Brain volume relationships
plt.subplot(2, 3, 3)
sns.scatterplot(data=df_viz, x='eTIV', y='nWBV', hue='CDR', alpha=0.7)
plt.title('Brain Volume Relationship')
plt.xlabel('Estimated Total Intracranial Volume')
plt.ylabel('Normalized Whole Brain Volume')

# Education vs SES
plt.subplot(2, 3, 4)
sns.scatterplot(data=df_viz, x='Educ', y='SES', hue='CDR', alpha=0.7)
plt.title('Education vs Socioeconomic Status')
plt.xlabel('Education (Years)')
plt.ylabel('Socioeconomic Status')

# MMSE distribution by CDR
plt.subplot(2, 3, 5)
sns.violinplot(data=df_viz, x='CDR', y='MMSE')
plt.title('MMSE Score Distribution by CDR')
plt.xlabel('CDR (Dementia Rating)')
plt.ylabel('MMSE Score')

# Age distribution
plt.subplot(2, 3, 6)
sns.histplot(data=df_viz, x='Age', hue='CDR', multiple='stack', alpha=0.7)
plt.title('Age Distribution by CDR')
plt.xlabel('Age')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# 3. Anomaly and Outlier Detection
plt.figure(figsize=(18, 12))

# Prepare data for anomaly detection
df_clean = df_viz.copy()
df_clean['M/F'] = df_clean['M/F'].apply(lambda x: 1 if x == 'M' else 0)

# Drop rows with missing CDR values (our target variable)
df_clean = df_clean.dropna(subset=['CDR'])

# Fill missing values with median for numeric columns
numeric_columns = ['Educ', 'SES', 'MMSE']
for col in numeric_columns:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Select features for anomaly detection
features_for_anomaly = ['M/F', 'Age', 'Educ', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF']
X_anomaly = df_clean[features_for_anomaly]

print(f"Data shape after cleaning: {X_anomaly.shape}")
print(f"Features used: {features_for_anomaly}")

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_anomaly)

# 1. Isolation Forest for Anomaly Detection
iso_forest = IsolationForest(contamination=0.1, random_state=42)
anomaly_labels = iso_forest.fit_predict(X_scaled)
df_clean['Anomaly'] = anomaly_labels

# Count anomalies
normal_count = np.sum(anomaly_labels == 1)
anomaly_count = np.sum(anomaly_labels == -1)

print(f"Normal samples: {normal_count}")
print(f"Anomalies detected: {anomaly_count}")
print(f"Anomaly percentage: {(anomaly_count / len(anomaly_labels)) * 100:.2f}%")

# Subplot 1: Anomaly Distribution
plt.subplot(3, 4, 1)
anomaly_counts = pd.Series(anomaly_labels).value_counts()
labels = ['Normal', 'Anomaly']
colors = ['lightgreen', 'red']
plt.pie(anomaly_counts.values, labels=labels, autopct='%1.1f%%', colors=colors)
plt.title('Anomaly Detection Results')

# Subplot 2: Age vs MMSE with anomalies highlighted
plt.subplot(3, 4, 2)
normal_data = df_clean[df_clean['Anomaly'] == 1]
anomaly_data = df_clean[df_clean['Anomaly'] == -1]
plt.scatter(normal_data['Age'], normal_data['MMSE'], c='blue', alpha=0.6, label='Normal', s=30)
plt.scatter(anomaly_data['Age'], anomaly_data['MMSE'], c='red', alpha=0.8, label='Anomaly', s=50)
plt.xlabel('Age')
plt.ylabel('MMSE Score')
plt.title('Age vs MMSE (Anomalies Highlighted)')
plt.legend()

# Subplot 3: Boxplot for Age outliers
plt.subplot(3, 4, 3)
sns.boxplot(data=df_clean, y='Age')
plt.title('Age Distribution - Outlier Detection')
plt.ylabel('Age')

# Subplot 4: Boxplot for MMSE outliers
plt.subplot(3, 4, 4)
sns.boxplot(data=df_clean, y='MMSE')
plt.title('MMSE Score - Outlier Detection')
plt.ylabel('MMSE Score')

# Subplot 5: Brain volume outliers
plt.subplot(3, 4, 5)
sns.boxplot(data=df_clean, y='eTIV')
plt.title('eTIV - Outlier Detection')
plt.ylabel('eTIV')

# Subplot 6: Normalized brain volume outliers
plt.subplot(3, 4, 6)
sns.boxplot(data=df_clean, y='nWBV')
plt.title('nWBV - Outlier Detection')
plt.ylabel('nWBV')

# Subplot 7: Education outliers
plt.subplot(3, 4, 7)
sns.boxplot(data=df_clean, y='Educ')
plt.title('Education - Outlier Detection')
plt.ylabel('Education Years')

# Subplot 8: SES outliers
plt.subplot(3, 4, 8)
sns.boxplot(data=df_clean, y='SES')
plt.title('SES - Outlier Detection')
plt.ylabel('Socioeconomic Status')

# Subplot 9: Brain volume relationship with anomalies
plt.subplot(3, 4, 9)
plt.scatter(normal_data['eTIV'], normal_data['nWBV'], c='blue', alpha=0.6, label='Normal', s=30)
plt.scatter(anomaly_data['eTIV'], anomaly_data['nWBV'], c='red', alpha=0.8, label='Anomaly', s=50)
plt.xlabel('eTIV')
plt.ylabel('nWBV')
plt.title('Brain Volume Relationship (Anomalies)')
plt.legend()

# Subplot 10: CDR distribution among anomalies
plt.subplot(3, 4, 10)
anomaly_cdr = anomaly_data['CDR'].value_counts()
normal_cdr = normal_data['CDR'].value_counts()
x_pos = np.arange(len(anomaly_cdr.index))
width = 0.35
plt.bar(x_pos - width/2, [normal_cdr.get(i, 0) for i in anomaly_cdr.index], 
        width, label='Normal', alpha=0.7)
plt.bar(x_pos + width/2, anomaly_cdr.values, width, label='Anomaly', alpha=0.7)
plt.xlabel('CDR')
plt.ylabel('Count')
plt.title('CDR Distribution: Normal vs Anomalies')
plt.xticks(x_pos, anomaly_cdr.index)
plt.legend()

# Subplot 11: Feature importance for anomaly detection
plt.subplot(3, 4, 11)
feature_means_normal = normal_data[features_for_anomaly].mean()
feature_means_anomaly = anomaly_data[features_for_anomaly].mean()
x_pos = np.arange(len(features_for_anomaly))
width = 0.35
plt.bar(x_pos - width/2, feature_means_normal, width, label='Normal', alpha=0.7)
plt.bar(x_pos + width/2, feature_means_anomaly, width, label='Anomaly', alpha=0.7)
plt.xlabel('Features')
plt.ylabel('Mean Value')
plt.title('Feature Means: Normal vs Anomalies')
plt.xticks(x_pos, features_for_anomaly, rotation=45)
plt.legend()

# Subplot 12: ASF outliers
plt.subplot(3, 4, 12)
sns.boxplot(data=df_clean, y='ASF')
plt.title('ASF - Outlier Detection')
plt.ylabel('Atlas Scaling Factor')

plt.tight_layout()
plt.show()

In [ ]:
# 4. Test both models with sample data
# No need for cv2, we'll use keras image preprocessing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import os
import glob

print("=" * 60)
print("TESTING BOTH MODELS WITH SAMPLE DATA")
print("=" * 60)

# Test CSV Model
print("\n1. TESTING CSV MODEL:")
print("-" * 30)

# Load the CSV model
csv_model = joblib.load('alzheimers_csv_model.joblib')

# Get a random sample from the test data
sample_idx = np.random.randint(0, len(X_test))
sample_data = X_test.iloc[sample_idx:sample_idx+1]
actual_label = y_test.iloc[sample_idx]

print(f"Sample data (row {sample_idx}):")
print(sample_data)
print(f"\nActual label: {actual_label} ({'Non-Demented' if actual_label == 0 else 'Demented'})")

# Make prediction
csv_prediction = csv_model.predict(sample_data)[0]
csv_probability = csv_model.predict_proba(sample_data)[0]

print(f"CSV Model Prediction: {csv_prediction} ({'Non-Demented' if csv_prediction == 0 else 'Demented'})")
print(f"Prediction Probabilities: Non-Demented={csv_probability[0]:.3f}, Demented={csv_probability[1]:.3f}")

# Test Image Model
print("\n\n2. TESTING IMAGE MODEL:")
print("-" * 30)

# Load the image model
image_model = load_model('alzheimers_image_model.h5')

# Get a random image from the dataset
random_class = np.random.choice(classes)
random_class_path = class_paths[random_class]

print(f"Testing with class: {random_class}")
print(f"Class path: {random_class_path}")

if os.path.isdir(random_class_path):
    # Get all image files
    image_extensions = ['*.png', '*.jpg', '*.jpeg']
    image_files_test = []
    for ext in image_extensions:
        image_files_test.extend(glob.glob(os.path.join(random_class_path, ext)))
    
    if image_files_test:
        random_image_file = np.random.choice(image_files_test)
        
        print(f"Testing image: {os.path.basename(random_image_file)}")
        print(f"Actual class: {random_class}")
        print(f"Image path: {random_image_file}")
        
        # Load and preprocess the image
        img = image.load_img(random_image_file, target_size=(128, 128))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        
        # Make prediction
        image_predictions = image_model.predict(img_array, verbose=0)
        predicted_class_idx = np.argmax(image_predictions[0])
        predicted_class = classes[predicted_class_idx]
        confidence = image_predictions[0][predicted_class_idx]
        
        print(f"\nImage Model Prediction: {predicted_class}")
        print(f"Confidence: {confidence:.3f}")
        print(f"All class probabilities:")
        for i, class_name in enumerate(classes):
            print(f"  {class_name}: {image_predictions[0][i]:.3f}")
        
        # Display the test image
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.title(f'Test Image\nActual: {random_class}\nPredicted: {predicted_class}')
        plt.axis('off')
        
        # Show prediction probabilities
        plt.subplot(1, 2, 2)
        bars = plt.bar(classes, image_predictions[0])
        plt.title('Prediction Probabilities')
        plt.xlabel('Classes')
        plt.ylabel('Probability')
        plt.xticks(rotation=45)
        
        # Highlight the predicted class
        bars[predicted_class_idx].set_color('red')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\nPrediction Result: {'✓ CORRECT' if predicted_class == random_class else '✗ INCORRECT'}")
    else:
        print("No image files found in the selected class directory")
        print("Available files:", os.listdir(random_class_path)[:5] if os.path.isdir(random_class_path) else "Directory not found")
else:
    print(f"Directory not found: {random_class_path}")

print("\n" + "=" * 60)
print("MODEL TESTING COMPLETED")
print("=" * 60)